In [17]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
class Network:
    def __init__(self):
        self._classes_count = 0
        self._learning_rate = 0
        self._max_iters = 0
        
        self._weights_hidden = []
        self._bias_hidden = 0
        self._weights_output = []
        self._bias_output = 0
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    def softmax(x):
        z = np.exp(x)
        return z / z.sum(axis=1, keepdims=True)
    
    def _one_hot_matrix(self, y):
        Y = np.zeros((y.shape[0], self._classes_count))
        for i in range(y.shape[0]):
            Y[i, y[i]] = 1
        return Y
    
    def fit(self, X, y, hidden_layer_count=5, classes_count=None, learning_rate=1e-5, max_iters=1e5):  
        
        self._classes_count = np.unique(y).shape[0] if classes_count is None else classes_count 
        self._learning_rate = learning_rate
        self._max_iters = int(max_iters)
        
        np.random.seed(1709)     
        
        self._weights_hidden = np.random.randn(X.shape[1], hidden_layer_count)
        self._bias_hidden = np.random.randn(hidden_layer_count)
        
        self._weights_output = np.random.randn(hidden_layer_count, self._classes_count)
        self._bias_output = np.random.randn(self._classes_count)
        
        y = self._one_hot_matrix(y)
        
        for i in range(self._max_iters + 1):                
            # forward propagate
            hidden_result = X @ self._weights_hidden + self._bias_hidden
            hidden_result = Network.sigmoid(hidden_result)
            
            output_result = hidden_result @ self._weights_output + self._bias_output
            output_result = Network.softmax(output_result)
            
            # backward propagate
            predict_error = output_result - y
            self._weights_output -= hidden_result.T @ predict_error * self._learning_rate
            self._bias_output -= predict_error.sum() * self._learning_rate
            
            hidden_error = predict_error @ self._weights_output.T * hidden_result * (1 - hidden_result)  
            self._weights_hidden -= X.T @ hidden_error * self._learning_rate            
            self._bias_hidden -= hidden_error.sum() * self._learning_rate
            
            
            if i % (self._max_iters / 10) == 0:
                print(f"Progress: {i / self._max_iters:.2%}, Error: {(predict_error ** 2).mean()}")
            
        
    def predict(self, X):
        hidden_result = X @ self._weights_hidden + self._bias_hidden
        hidden_result = Network.sigmoid(hidden_result)

        output_result = hidden_result @ self._weights_output + self._bias_output
        output_result = Network.softmax(output_result)
        
        return np.argmax(output_result, axis=1)
    
    def score(self, X, y):
        return (self.predict(X) == y).mean()

## 1. Пример с классификацией вида ириса

In [3]:
df = pd.read_csv("iris.csv", header=None, names=[
    "Sepal length (cm)",
    "Sepal width (cm)",
    "Petal length (cm)",
    "Petal width (cm)",
    "Species"
])
df["Species"] = df["Species"].astype("category").cat.codes
df.head()

,Sepal length (cm),Sepal width (cm),Petal length (cm),Petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
X = df.to_numpy()[:, :-1]
y = df.to_numpy()[:, -1].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1709)

### Собственная реализация

In [5]:
model = Network()
%time model.fit(X_train, y_train, hidden_layer_count=7)

Progress: 0.00%, Error: 0.3890498660139191
Progress: 10.00%, Error: 0.11944896382702844
Progress: 20.00%, Error: 0.11329075402730386
Progress: 30.00%, Error: 0.11048642969240076
Progress: 40.00%, Error: 0.0935594141380361
Progress: 50.00%, Error: 0.07719735474239084
Progress: 60.00%, Error: 0.05940532120366936
Progress: 70.00%, Error: 0.04492328051716258
Progress: 80.00%, Error: 0.03517442103560297
Progress: 90.00%, Error: 0.02900750347894271
Progress: 100.00%, Error: 0.025067433666509962
CPU times: user 8.4 s, sys: 12.9 ms, total: 8.42 s
Wall time: 8.45 s


In [6]:
print(f"Score: {model.score(X_test, y_test)}")

Score: 0.9666666666666667


### TensorFlow нейронная сеть

In [7]:
model = tf.keras.Sequential([
    keras.layers.Dense(7, activation='sigmoid'),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
%time model.fit(X_train, y_train, epochs=1000, verbose=0)

CPU times: user 6.16 s, sys: 313 ms, total: 6.48 s
Wall time: 5.59 s


In [9]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)

print(f"Score: {acc}")

Score: 0.9666666388511658


## 2. Пример с классификацией цифр

In [10]:
df = pd.read_csv("digits.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
part = df.to_numpy()[:10000]

X = part[:, 1:] / 256
y = part[:, 0].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1709)

### Собственная реализация

In [12]:
model = Network()
%time model.fit(X_train, y_train, hidden_layer_count=10, classes_count=10, learning_rate=1e-4)

Progress: 0.00%, Error: 0.1463437600309986
Progress: 10.00%, Error: 0.006674574183539785
Progress: 20.00%, Error: 0.004266162520365379
Progress: 30.00%, Error: 0.0032102583128008964
Progress: 40.00%, Error: 0.0025866600063945704
Progress: 50.00%, Error: 0.0020958441714404854
Progress: 60.00%, Error: 0.0018197434530583543
Progress: 70.00%, Error: 0.0016608303217026863
Progress: 80.00%, Error: 0.00146440424185274
Progress: 90.00%, Error: 0.0013323785012454015
Progress: 100.00%, Error: 0.0012034678228233318
CPU times: user 3h 14min 20s, sys: 30min 10s, total: 3h 44min 30s
Wall time: 1h 7min 41s


In [13]:
model.score(X_test, y_test)

0.858

### TensorFlow нейронная сеть

In [14]:
model = tf.keras.Sequential([
    keras.layers.Dense(10, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
%time model.fit(X_train, y_train, epochs=1000, verbose=0)

CPU times: user 6min 5s, sys: 23.5 s, total: 6min 28s
Wall time: 4min 3s


In [16]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)

print(f"Score: {acc}")

Score: 0.8615000247955322
